# SynapticRAG Basic Usage

This notebook demonstrates the basic usage of SynapticRAG, a hybrid Retrieval-Augmented Generation system that combines graph-based retrieval with memory-inspired knowledge discovery.

In [ ]:
import os
import sys
from dotenv import load_dotenv

# Add parent directory to path
sys.path.append(os.path.abspath('..'))

# Load environment variables
load_dotenv()

from synaptic.pipeline import SynapticPipeline
from synaptic.config import SynapticConfig

## Configuration

First, let's create a configuration for our pipeline. We'll use the default settings but you can customize them based on your needs.

In [ ]:
config = SynapticConfig(
    working_dir='./workspace',
    
    # Memory settings
    memory={
        'model_name_or_path': 'anthropic/claude-3-sonnet-20240229',
        'load_in_4bit': True
    },
    
    # Graph settings
    graph={
        'chunk_token_size': 1200,
        'gnn_hidden_channels': 256
    },
    
    # Retriever settings
    retriever={
        'model_name_or_path': 'BAAI/bge-large-en-v1.5',
        'hits': 3
    }
)

# Initialize pipeline
pipeline = SynapticPipeline(config=config)

## Sample Documents

Let's create some sample documents to work with.

In [ ]:
documents = [
    """
    Neural networks are a set of algorithms, modeled loosely after the human brain, 
    that are designed to recognize patterns. They interpret sensory data through a 
    kind of machine perception, labeling or clustering raw input. The patterns they 
    recognize are numerical, contained in vectors, into which all real-world data, 
    be it images, sound, text or time series, must be translated.
    """,
    """
    Graph Neural Networks (GNNs) are a class of deep learning methods designed to 
    perform inference on data described by graphs. GNNs are neural networks that can 
    be directly applied to graphs, and provide an easy way to do node-level, 
    edge-level, and graph-level prediction tasks.
    """,
    """
    PyTorch Geometric (PyG) is a library built upon PyTorch to easily write and 
    train Graph Neural Networks (GNNs) for a wide range of applications related to 
    structured data. It consists of various methods for deep learning on graphs and 
    other irregular structures.
    """
]

# Process documents
stats = pipeline.process_documents(documents)
print("Processing Statistics:")
print(f"Number of documents: {stats.num_documents}")
print(f"Number of entities: {stats.num_entities}")
print(f"Number of relationships: {stats.num_relationships}")

## Simple Queries

Let's try some simple queries to test the system.

In [ ]:
# Direct question
query = "What are neural networks?"
result = pipeline.query(query)
print("Query:", query)
print("\nResponse:", result.response)
print("\nSources:", result.sources)

## Complex Queries

Now let's try some more complex queries that require combining information from multiple documents.

In [ ]:
# Complex query requiring information synthesis
query = "How do Graph Neural Networks relate to traditional neural networks, and how can PyTorch Geometric help implement them?"
result = pipeline.query(query)
print("Query:", query)
print("\nResponse:", result.response)
print("\nSources:", result.sources)

## Memory Analysis

Let's examine how the system uses its memory to handle queries.

In [ ]:
# Get memory clues for a query
query = "What tools are available for implementing graph neural networks?"
result = pipeline.query(query, return_sources=True)

print("Query:", query)
print("\nGenerated Clues:")
for clue in result.metadata['clues'].split('\n'):
    print(f"- {clue}")
print("\nResponse:", result.response)

## Graph Analysis

Finally, let's visualize the knowledge graph constructed from our documents.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Get the NetworkX graph
nx_graph = pipeline.rag.graph_builder.nx_graph

# Create plot
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(nx_graph)

# Draw nodes
nx.draw_networkx_nodes(nx_graph, pos, node_color='lightblue', 
                      node_size=2000, alpha=0.6)
nx.draw_networkx_labels(nx_graph, pos)

# Draw edges
nx.draw_networkx_edges(nx_graph, pos, alpha=0.5)
edge_labels = nx.get_edge_attributes(nx_graph, 'type')
nx.draw_networkx_edge_labels(nx_graph, pos, edge_labels)

plt.title("Knowledge Graph Visualization")
plt.axis('off')
plt.show()